In [61]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
import tensorflow as tf

#  Import and read the charity_data.csv.
application_df = pd.read_csv("https://static.bc-edx.com/data/dla-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [62]:
#Identify the target and feature variables 
target_variables = application_df['IS_SUCCESSFUL']
feature_variables = application_df.loc[:, ~application_df.columns.isin(['IS_SUCCESSFUL', 'EIN', 'NAME'])]

In [63]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=['EIN', 'NAME', 'ORGANIZATION'])
application_df.head()


,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,1,100000-499999,N,142590,1


In [64]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [65]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64

In [66]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = ['T9', 'T13', 'T12', 'T2', 'T25', 'T14', 'T29', 'T15', 'T17']

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure replacement was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64

In [67]:
# Look at CLASSIFICATION value counts to identify and replace with "Other"
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64

In [68]:
# You may find it helpful to look at CLASSIFICATION value counts >1
application_df['CLASSIFICATION'].value_counts()[(application_df['CLASSIFICATION'].value_counts() > 1)]

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: count, dtype: int64

In [69]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = pd.DataFrame(application_df['CLASSIFICATION'].value_counts()[(application_df['CLASSIFICATION'].value_counts() <= 70)]).index.to_list()

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure replacement was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
Other      499
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
Name: count, dtype: int64

In [70]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df = pd.get_dummies(application_df)

In [71]:
# Split our preprocessed data into our features and target arrays
target_array = application_df['IS_SUCCESSFUL']
feature_array = application_df.loc[:, ~application_df.columns.isin(['IS_SUCCESSFUL'])].values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(feature_array, target_array, train_size=0.7, random_state=42)

In [72]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [73]:
# Add additional libraries to optimize model construction
from tensorflow.keras import layers, regularizers, callbacks
from sklearn.model_selection import KFold
from imblearn.over_sampling import SMOTE

In [74]:
# Apply SMOTE to handle any class imbalance
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train_scaled, y_train)

In [75]:
# Define the model with added regularization and increased complexity
def create_model():
    model = tf.keras.Sequential([
        layers.Dense(128, input_dim=X_resampled.shape[1], activation='relu', kernel_regularizer=regularizers.l2(0.003)),
        layers.BatchNormalization(),
        layers.Dropout(0.4),
        layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.003)),
        layers.BatchNormalization(),
        layers.Dropout(0.4),
        layers.Dense(32, activation='relu', kernel_regularizer=regularizers.l2(0.003)),
        layers.Dropout(0.4),
        layers.Dense(1, activation='sigmoid')
    ])

    # Compile the model with a learning rate scheduler
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.003),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    return model

In [76]:
# Implement early stopping
early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [77]:
# Implement learning rate scheduler
lr_scheduler = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5)

In [78]:
# K-Fold Cross-Validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)
fold_no = 1
val_accuracies = []

for train_index, val_index in kf.split(X_resampled):
    X_train_fold, X_val_fold = X_resampled[train_index], X_resampled[val_index]
    y_train_fold, y_val_fold = y_resampled[train_index], y_resampled[val_index]

    # Create a new model instance
    model = create_model()

    # Train the model with a modified batch size
    history = model.fit(X_train_fold, y_train_fold,
                        epochs=100,
                        batch_size=32, 
                        validation_data=(X_val_fold, y_val_fold),
                        callbacks=[early_stopping, lr_scheduler],
                        verbose=2)

Epoch 1/100


c:\Users\tammy\OneDrive\Documents\GitHub\Deep-Learning-Challenge\venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


638/638 - 4s - 6ms/step - accuracy: 0.6869 - loss: 0.8909 - val_accuracy: 0.7155 - val_loss: 0.6755 - learning_rate: 0.0030
Epoch 2/100
638/638 - 1s - 2ms/step - accuracy: 0.7134 - loss: 0.6552 - val_accuracy: 0.7124 - val_loss: 0.6129 - learning_rate: 0.0030
Epoch 3/100
638/638 - 1s - 2ms/step - accuracy: 0.7163 - loss: 0.6310 - val_accuracy: 0.7180 - val_loss: 0.6107 - learning_rate: 0.0030
Epoch 4/100
638/638 - 1s - 2ms/step - accuracy: 0.7150 - loss: 0.6337 - val_accuracy: 0.7200 - val_loss: 0.6221 - learning_rate: 0.0030
Epoch 5/100
638/638 - 1s - 2ms/step - accuracy: 0.7175 - loss: 0.6296 - val_accuracy: 0.7202 - val_loss: 0.6151 - learning_rate: 0.0030
Epoch 6/100
638/638 - 1s - 2ms/step - accuracy: 0.7148 - loss: 0.6275 - val_accuracy: 0.7145 - val_loss: 0.6094 - learning_rate: 0.0030
Epoch 7/100
638/638 - 1s - 2ms/step - accuracy: 0.7157 - loss: 0.6255 - val_accuracy: 0.7212 - val_loss: 0.6148 - learning_rate: 0.0030
Epoch 8/100
638/638 - 1s - 2ms/step - accuracy: 0.7175 - los

c:\Users\tammy\OneDrive\Documents\GitHub\Deep-Learning-Challenge\venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


638/638 - 4s - 6ms/step - accuracy: 0.6901 - loss: 0.8828 - val_accuracy: 0.7168 - val_loss: 0.6792 - learning_rate: 0.0030
Epoch 2/100
638/638 - 2s - 3ms/step - accuracy: 0.7121 - loss: 0.6518 - val_accuracy: 0.7182 - val_loss: 0.6260 - learning_rate: 0.0030
Epoch 3/100
638/638 - 1s - 2ms/step - accuracy: 0.7116 - loss: 0.6327 - val_accuracy: 0.7227 - val_loss: 0.6168 - learning_rate: 0.0030
Epoch 4/100
638/638 - 1s - 2ms/step - accuracy: 0.7126 - loss: 0.6293 - val_accuracy: 0.7215 - val_loss: 0.6168 - learning_rate: 0.0030
Epoch 5/100
638/638 - 1s - 2ms/step - accuracy: 0.7136 - loss: 0.6301 - val_accuracy: 0.7109 - val_loss: 0.6213 - learning_rate: 0.0030
Epoch 6/100
638/638 - 1s - 2ms/step - accuracy: 0.7098 - loss: 0.6294 - val_accuracy: 0.7152 - val_loss: 0.6156 - learning_rate: 0.0030
Epoch 7/100
638/638 - 1s - 2ms/step - accuracy: 0.7135 - loss: 0.6257 - val_accuracy: 0.7137 - val_loss: 0.6185 - learning_rate: 0.0030
Epoch 8/100
638/638 - 1s - 2ms/step - accuracy: 0.7139 - los

...

638/638 - 4s - 6ms/step - accuracy: 0.6811 - loss: 0.9446 - val_accuracy: 0.7146 - val_loss: 0.7193 - learning_rate: 0.0030
Epoch 2/100
638/638 - 1s - 2ms/step - accuracy: 0.7151 - loss: 0.6677 - val_accuracy: 0.7115 - val_loss: 0.6359 - learning_rate: 0.0030
Epoch 3/100
638/638 - 1s - 2ms/step - accuracy: 0.7189 - loss: 0.6314 - val_accuracy: 0.7072 - val_loss: 0.6271 - learning_rate: 0.0030
Epoch 4/100
638/638 - 1s - 2ms/step - accuracy: 0.7147 - loss: 0.6273 - val_accuracy: 0.7117 - val_loss: 0.6266 - learning_rate: 0.0030
Epoch 5/100
638/638 - 1s - 2ms/step - accuracy: 0.7177 - loss: 0.6270 - val_accuracy: 0.7103 - val_loss: 0.6252 - learning_rate: 0.0030
Epoch 6/100
638/638 - 1s - 2ms/step - accuracy: 0.7149 - loss: 0.6262 - val_accuracy: 0.7099 - val_loss: 0.6211 - learning_rate: 0.0030
Epoch 7/100
638/638 - 1s - 2ms/step - accuracy: 0.7178 - loss: 0.6246 - val_accuracy: 0.7129 - val_loss: 0.6230 - learning_rate: 0.0030
Epoch 8/100
638/638 - 1s - 2ms/step - accuracy: 0.7154 - los

c:\Users\tammy\OneDrive\Documents\GitHub\Deep-Learning-Challenge\venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


638/638 - 3s - 5ms/step - accuracy: 0.6807 - loss: 0.9191 - val_accuracy: 0.7164 - val_loss: 0.7084 - learning_rate: 0.0030
Epoch 2/100
638/638 - 1s - 2ms/step - accuracy: 0.7155 - loss: 0.6603 - val_accuracy: 0.7190 - val_loss: 0.6246 - learning_rate: 0.0030
Epoch 3/100
638/638 - 1s - 2ms/step - accuracy: 0.7151 - loss: 0.6352 - val_accuracy: 0.7227 - val_loss: 0.6139 - learning_rate: 0.0030
Epoch 4/100
638/638 - 1s - 2ms/step - accuracy: 0.7145 - loss: 0.6297 - val_accuracy: 0.7168 - val_loss: 0.6166 - learning_rate: 0.0030
Epoch 5/100
638/638 - 1s - 2ms/step - accuracy: 0.7147 - loss: 0.6294 - val_accuracy: 0.7213 - val_loss: 0.6139 - learning_rate: 0.0030
Epoch 6/100
638/638 - 1s - 2ms/step - accuracy: 0.7119 - loss: 0.6280 - val_accuracy: 0.7213 - val_loss: 0.6172 - learning_rate: 0.0030
Epoch 7/100
638/638 - 1s - 2ms/step - accuracy: 0.7116 - loss: 0.6277 - val_accuracy: 0.7178 - val_loss: 0.6109 - learning_rate: 0.0030
Epoch 8/100
638/638 - 1s - 2ms/step - accuracy: 0.7149 - los

c:\Users\tammy\OneDrive\Documents\GitHub\Deep-Learning-Challenge\venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


638/638 - 4s - 6ms/step - accuracy: 0.6794 - loss: 0.9240 - val_accuracy: 0.7209 - val_loss: 0.6897 - learning_rate: 0.0030
Epoch 2/100
638/638 - 1s - 2ms/step - accuracy: 0.7139 - loss: 0.6592 - val_accuracy: 0.7245 - val_loss: 0.6127 - learning_rate: 0.0030
Epoch 3/100
638/638 - 1s - 2ms/step - accuracy: 0.7114 - loss: 0.6333 - val_accuracy: 0.7268 - val_loss: 0.6170 - learning_rate: 0.0030
Epoch 4/100
638/638 - 1s - 2ms/step - accuracy: 0.7135 - loss: 0.6309 - val_accuracy: 0.7233 - val_loss: 0.6020 - learning_rate: 0.0030
Epoch 5/100
638/638 - 1s - 2ms/step - accuracy: 0.7128 - loss: 0.6309 - val_accuracy: 0.7225 - val_loss: 0.6090 - learning_rate: 0.0030
Epoch 6/100
638/638 - 1s - 2ms/step - accuracy: 0.7113 - loss: 0.6306 - val_accuracy: 0.7211 - val_loss: 0.6165 - learning_rate: 0.0030
Epoch 7/100
638/638 - 1s - 2ms/step - accuracy: 0.7146 - loss: 0.6276 - val_accuracy: 0.7264 - val_loss: 0.6152 - learning_rate: 0.0030
Epoch 8/100
638/638 - 1s - 2ms/step - accuracy: 0.7137 - los

In [79]:
# Evaluate the model
val_loss, val_accuracy = model.evaluate(X_val_fold, y_val_fold, verbose=0)
print(f"Fold {fold_no} - Validation Accuracy: {val_accuracy}")
val_accuracies.append(val_accuracy)
fold_no += 1

Fold 1 - Validation Accuracy: 0.7209256887435913


In [80]:
# Calculate average accuracy across folds
average_val_accuracy = np.mean(val_accuracies)
print(f"Average Validation Accuracy: {average_val_accuracy}")

Average Validation Accuracy: 0.7209256887435913


In [81]:
# Export model to HDF5 file
model.save('AlphabetSoup_OpExTrials_T5.h5')